# Generate Market Book by Limit (MBL) limits and trades using dedicated service [Incremental Updates] - Python

### Overview
Market Book by Price (MBP) also known as Market Book by Limit (MBL), is the price-based data of the book.
MBL restricts updates to a maximum of specified depth price levels and consolidates all the quantity (size) into a single update for each price level, which includes the total quantity (size).

**Note** : This sample is not about Market Book by Order (MBO). For the MBO please refer the notebook **[Tick] mbo_quotes**

Ganymede can translate streams in the MBO (Market By Order) format to MBL/MBP (Market By Limit, or Market By Price) order book data. 
In this example we use two well known ICE data sources for Euronext
1. One contributes full order book in MBO format (e.g: flat quotes)
2. The other contributes trades

The MBO flat quoted data is on-the-fly translated to the more easily usable MBL/MBP data (matrix)

For more information on MBL/MBP and MBO you can for example read [here](https://www.cmegroup.com/education/market-by-order-mbo.html)

### Inputs/outputs
Trades and book extraction sample requires several inputs :
* instrument identifier
* book depth 
* book update type: in this use-case, the update type is **first snapshot then incremental updates**.
* date and time constraints

The service returns the trades and books for the given instrument.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and book tick data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    
***

# Run MBL incremental updates sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import google.protobuf.wrappers_pb2 as wrappers
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.identifier_and_level_pb2 as identifier_and_level
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.type.shared.v1.book_updates_pb2 as book_updates
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2 as tick_trades_and_book
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2_grpc as tick_trades_and_book_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades and book* service, we need to specify:
* Instrument identifier
* Time period selection
* Depth
* Book update type

#### 3.1 Instrument selection

In [ ]:
# Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
# The ICE ticker
ticker = 'E:BNP'

# The book limit : get the top 10 best limits
my_max_depth = 10

# Select bookupdates type  BOOK_UPDATES_SNAPSHOTS_ONLY or BOOK_UPDATES_INCREMENTALS
book_updates_type = book_updates.BOOK_UPDATES_INCREMENTALS

#### 3.2 Time period delimitation

In [ ]:
# Create time intervals (we are using Google date format)
# Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 11, day = 1), 
    end_date = date.Date(year = 2021, month = 11, day = 1)
)

# Build the tick quotes request time interval (we are using Google date time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 30, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.3 Request creation

In [ ]:
#this source is trades only Euronext
# this source is MBO/FOB Euronext
my_provider = 'ICE'
identifier_1 = identifier_and_level.IdentifierAndLevel(ticker = ticker,exchange = '787',level = level.LEVEL_TRADES, provider = wrappers.StringValue(value=my_provider)  )
identifier_2 = identifier_and_level.IdentifierAndLevel(ticker = ticker, exchange = 'EQUITY_L2_973', level = level.LEVEL_TRADES_AND_BOOK, provider = wrappers.StringValue(value=my_provider) )
my_identifiers = [identifier_1, identifier_2]


In [ ]:
# generate the tick trades and book request
request = tick_trades_and_book.TickTradesAndBookRequest(
    identifiers = my_identifiers,
    constraints = my_constraints,
    book_updates = book_updates_type,
    max_depth = wrappers.Int32Value(value = my_max_depth),
    adjustment = False
)

#### 3.4 Request processing and result export


In the following code snippet, we request trades and book for the given instrument.
The streamed response is exported on the fly in a file.

In [ ]:
from datetime import timedelta

books_count,trades_count,mappings_count = 0,0,0
filename = '{0}_mbl_incremental_updates.txt'.format(ticker)
with open(filename, mode='w') as file:
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:  

            # instantiate the tick quotes service
            service = tick_trades_and_book_service.TickTradesAndBookServiceStub(channel)

            # process the request
            metadata = [('authorization', token)]
            for current in service.TickTradesAndBook(request=request, metadata=metadata):
                
                if current.data.time_stamp.seconds > 0: # handling books/trades separately from first line (asset mapping)
                    time =datetime.fromtimestamp(current.data.time_stamp.seconds) + timedelta(microseconds = current.data.time_stamp.nanos/1000)
                    
                    if len(current.data.book.bid) > 0 or len(current.data.book.ask) > 0: # handling books
                        books_count+=1
                        book = current.data.book
                        str_mapping = current.data.mapping
                        file.write(('[BOOK]  {0} [{1}]'.format(time, str_mapping))+'\n')
                        book_max = max(len(book.bid), len(book.ask))
                        for i in range(0,book_max):
                            bid_depth,ask_depth,bid_price,ask_price,bid_size,ask_size = '','','','','',''
                            if i< len(book.bid):
                                bid_price = book.bid[i].price
                                bid_size = book.bid[i].size
                                bid_depth = book.bid[i].depth
                            if i< len(book.ask):
                                ask_price = book.ask[i].price
                                ask_size = book.ask[i].size
                                ask_depth = book.ask[i].depth
                            my_str = '{0:10} {1:10} {2:10} {3:10} {4:10} {5:10}'.format(bid_depth, bid_size, bid_price, ask_price, ask_size, ask_depth)
                            file.write((my_str)+'\n')
                        file.write('\n')

                    else: # handling trade
                        trades_count+=1
                        trade = current.data.trade
                        str_mapping = current.data.mapping
                        time =datetime.fromtimestamp(current.data.time_stamp.seconds) + timedelta(microseconds = current.data.time_stamp.nanos/1000)
                        str_price = trade.price
                        str_size = trade.size
                        str_condition = trade.condition
                        str_id = trade.id
                        file.write(('[TRADE] {0} [{1}]'.format(time, str_mapping))+'\n')
                        file.write(('{0}@{1} Id={2} Condition={3}'.format(str_size, str_price, str_id, str_condition))+'\n\n')
                    
                else:  
                    mappings_count+=1
                    file.write(('[MAPPING]')+'\n')
                    for i in range(len(current.mapping.values)):
                        str_provider = current.mapping.values[i].identifier.provider.value
                        str_exchange = current.mapping.values[i].identifier.exchange
                        str_ticker = current.mapping.values[i].identifier.ticker
                        my_str = '# [{0}] => {1}/{2}/{3}'.format(i, str_provider, str_exchange, str_ticker)
                        file.write((my_str)+'\n')
                    file.write('\n')
                    
                    
                    
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())
        
# Some display
print('----- Done exporting ----- ')
print('Mappings: {}'.format(mappings_count))
print('Trades: {}'.format(trades_count))
print('Books: {}'.format(books_count))
print('Check export in {}'.format(filename)) 